In [61]:
import numpy as np 
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from spacy.symbols import ORTH
from collections import Counter
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import numpy as np
from collections import defaultdict
import re
from sklearn.metrics import confusion_matrix

## Read file into DataFrame

In [2]:
# read the file into DataFrame
df = pd.read_csv('./CAMEO_IDEA_labeled_data.csv')

# separate content and label
text = df['Content']
labels = df['Category Code']

## Tokenize the text

In [3]:
# funtion tokenize sentence
tokenizer = spacy.load("en_core_web_sm")
stopwords = tokenizer.Defaults.stop_words
# tokenize, lemmatize the text, drop punctuations and stopwords
tokenize = lambda t: [token.lemma_ for token in tokenizer(t) if (not token.is_punct) and (not token.is_stop)]

# only tokenize the text
# tokenize = lambda t: [token.text for token in tokenizer(t)]

In [4]:
# build dictionary <key=word : value=count>
cnt = Counter()
size = text.size
for idx in range(size):
    for word in tokenize(text[idx]):
        cnt[word] += 1

In [5]:
# filter out low-frequency word
min_threshold = 1
count = {x: count for x, count in cnt.items() if count >= min_threshold}

In [6]:
# filter out high-frequency word
min_threshold = 1
count = {x: count for x, count in cnt.items() if count <= min_threshold}

## Split dataset into train set and test set

In [7]:
X = np.array(text)
y = np.array(labels)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

## Prepare for word embedding

In [8]:
# download glove dictionary
# def download_glove():
#     ! wget http://nlp.stanford.edu/data/glove.6B.zip
#     ! unzip glove.6B.zip -C data
    
# download_glove()
# ! unzip glove.6B.zip

In [9]:
# load word embedding dictionary (<key=word : value=vector>)
def load_embedding_dict():
    embeddings_dict = {}
    with open("glove.6B.50d.txt", 'r') as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            embeddings_dict[word] = vector
    return embeddings_dict

glove_dic = load_embedding_dict()

In [10]:
# create dictionaries(<key=word : value=index number>) (<key=word : value=vector>)
def create_embedding_matrix(count,emb_size=50):
    size = len(count) + 2
    word_idx_dict = {}
    word_vec = np.zeros((size, emb_size), dtype="float32")
    
    # add padding and UNK keyword
    word_idx_dict[""] = 0
    word_vec[0] = np.zeros(emb_size, dtype='float32')
    word_idx_dict["UNK"] = 1
    word_vec[1] = np.random.uniform(-0.25, 0.25, emb_size)

    for i, word in enumerate(count.keys()):
        word_idx_dict[word] = i + 2

        if word in glove_dic:
            word_vec[i + 2] = glove_dic[word]
        else:
            word_vec[i + 2] = np.random.uniform(-0.25,0.25, emb_size)

    return word_idx_dict, word_vec
    
word_idx_dict, pretrained_weight = create_embedding_matrix(count)

## Prepare for encoding sentence

In [11]:
def encode_sentence(line, word_idx_dict, N=400, padding_start=True):
    tokens = tokenize(line)
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([word_idx_dict.get(word, word_idx_dict["UNK"]) for word in tokens])
    length = min(N, len(enc1))
    if padding_start:
        enc[:length] = enc1[:length]
    else:
        enc[N - length:] = enc1[:length]
    return enc, length

## Build DataSet and DataLoader for model

In [33]:
class EventDataset(Dataset):
    def __init__(self, X, y, N=40, padding_start=False):
        self.y = y
        self.X = [encode_sentence(line, word_idx_dict, N, padding_start) for line in X]
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x, s = self.X[idx]
        return x, s, self.y[idx]

In [34]:
train_ds = EventDataset(X_train, y_train)
valid_ds = EventDataset(X_val, y_val)
train_dl = DataLoader(train_ds, batch_size=30, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=30)

In [98]:
def update_optimizer(optimizer, lr):
    for i, param_group in enumerate(optimizer.param_groups):
        param_group["lr"] = lr

In [99]:

def train_epocs(model, optimizer, train_dl, val_dl, epochs=10):
    global max_acc
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            # s is not used in this model
            x = x.long() 
            y = y.long() 
#             x = x.long()
#             y = y.float()
            y_pred = model(x)
            optimizer.zero_grad()
            
#             print(y.size())
#             print(y_pred.squeeze(0).size())
            loss = F.cross_entropy(y_pred.squeeze(0), y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, val_dl)
        if val_acc > max_acc:
            max_acc = val_acc
        if i % 5 == 1:
            print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

In [107]:
def val_metrics(model, val_dl):
    model.eval()
    total = 0
    sum_loss = 0
    correct = 0
    for x, s, y in val_dl:
        x = x.long()  #.cuda()
        y = y.long()
        batch = y.shape[0]
        out = model(x)
        
        loss = F.cross_entropy(out.squeeze(0), y)
        sum_loss += batch*(loss.item())
        total += batch
        _, pred = torch.max(out.squeeze(0), 1) 
        
#         print(pred.size())
        correct += (pred == y.data).float().sum().item()
    val_loss = sum_loss/total
    val_acc = correct/total
    return val_loss, val_acc

In [108]:
class LSTMModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights=None) :
        super(LSTMModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        if glove_weights is not None:
            self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
            self.embeddings.weight.requires_grad = True ## freeze embeddings
            
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 3)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
#         print(x.size())
        out_pack, ht = self.lstm(x)
        x = self.linear(ht[-1])
#         print(x.size())
        return x

In [109]:
for i in range(4): 
    vocab_size = len(word_idx_dict)
    model = LSTMModel(vocab_size, 50, 50*(i+1), glove_weights = pretrained_weight)
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=0.01)
    max_acc = 0
    train_epocs(model, optimizer, train_dl, valid_dl, epochs=30)
    print(max_acc)
    
    vocab_size = len(word_idx_dict)
    model = LSTMModel(vocab_size, 50, 50*(i+1), glove_weights = pretrained_weight)
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=0.1)
    max_acc = 0
    train_epocs(model, optimizer, train_dl, valid_dl, epochs=30)
    print(max_acc)

train loss 0.307 val loss 0.270 and val accuracy 0.899
train loss 0.070 val loss 0.324 and val accuracy 0.905
train loss 0.040 val loss 0.370 and val accuracy 0.880
train loss 0.038 val loss 0.352 and val accuracy 0.911
train loss 0.023 val loss 0.391 and val accuracy 0.899
train loss 0.023 val loss 0.368 and val accuracy 0.911
0.9177215189873418
train loss 0.253 val loss 0.250 and val accuracy 0.905
train loss 0.104 val loss 0.361 and val accuracy 0.854
train loss 0.088 val loss 0.488 and val accuracy 0.829
train loss 0.352 val loss 2.011 and val accuracy 0.918
train loss 0.345 val loss 1.089 and val accuracy 0.880
train loss 0.684 val loss 5.682 and val accuracy 0.842
0.9177215189873418
train loss 0.315 val loss 0.290 and val accuracy 0.899
train loss 0.056 val loss 0.324 and val accuracy 0.905
train loss 0.037 val loss 0.416 and val accuracy 0.905
train loss 0.027 val loss 0.368 and val accuracy 0.905
train loss 0.022 val loss 0.462 and val accuracy 0.918
train loss 0.032 val loss 0

KeyboardInterrupt: 

In [110]:
class LSTMModel1(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, hidden_dim1, glove_weights=None) :
        super(LSTMModel1,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        if glove_weights is not None:
            self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
            self.embeddings.weight.requires_grad = True ## freeze embeddings
            
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim1)
        self.linear1 = nn.Linear(hidden_dim1, 3)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
#         print(x.size())
        out_pack, ht = self.lstm(x)
        x = self.linear(ht[-1])
        x = self.linear1(x)
        return x

In [112]:
for i in range(4):
    model1 = LSTMModel1(vocab_size, 50, 50*(i+1), 50, glove_weights = pretrained_weight)
    parameters1 = filter(lambda p: p.requires_grad, model1.parameters())
    optimizer1 = torch.optim.Adam(parameters1, lr=0.01)
    max_acc = 0
    train_epocs(model1, optimizer1, train_dl, valid_dl, epochs=30)
    print(max_acc)
    
    model1 = LSTMModel1(vocab_size, 50, 50*(i+1), 50, glove_weights = pretrained_weight)
    parameters1 = filter(lambda p: p.requires_grad, model1.parameters())
    optimizer1 = torch.optim.Adam(parameters1, lr=0.1)
    max_acc = 0
    train_epocs(model1, optimizer1, train_dl, valid_dl, epochs=30)
    print(max_acc)

train loss 0.316 val loss 0.254 and val accuracy 0.899
train loss 0.170 val loss 0.230 and val accuracy 0.911
train loss 0.029 val loss 0.453 and val accuracy 0.911
train loss 0.031 val loss 0.380 and val accuracy 0.918
train loss 0.024 val loss 0.601 and val accuracy 0.899
train loss 0.023 val loss 0.424 and val accuracy 0.918
0.9240506329113924
train loss 3.523 val loss 1.507 and val accuracy 0.899
train loss 0.136 val loss 0.406 and val accuracy 0.861
train loss 0.074 val loss 0.370 and val accuracy 0.886
train loss 0.076 val loss 0.457 and val accuracy 0.886
train loss 0.049 val loss 0.395 and val accuracy 0.911
train loss 0.044 val loss 0.418 and val accuracy 0.905
0.9113924050632911
train loss 0.340 val loss 0.311 and val accuracy 0.899


KeyboardInterrupt: 